In [379]:
import json
import copy
import random
import numpy as np
import pandas as pd
import math

In [416]:
time_slots = range(0, 60)
k = 5

In [996]:
lecturers = pd.DataFrame(np.array([['Kartelj'], ['Cukic'], ['Mitic'], ['Malkov']]),
                   columns=['name'])
preffered_timeslots = [[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23],[],[],[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35]]
lecturers['preffered_timeslots'] = preffered_timeslots
not_preffered_timeslots = [[24,25,26,27,28,29,30,31,32,33,34,35],[],[],[36,37,38,39,40,41,42,43,44,45,46,47]]
lecturers['not_preffered_timeslots'] = not_preffered_timeslots
preffered_locations = [{'1', '2'},set(),set(),{'0'}]
lecturers['preffered_locations'] = preffered_locations
lecturers

,name,preffered_timeslots,not_preffered_timeslots,preffered_locations
0,Kartelj,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35]","{2, 1}"
1,Cukic,[],[],{}
2,Mitic,[],[],{}
3,Malkov,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47]",{0}


In [37]:
classes = pd.DataFrame(np.array([['Racunarske mreze', None, 2], ['Racunarske mreze - vezbe', 'racunari', 3], 
                                 ['Racunarska inteligencija', None, 2], ['Racunarska inteligencija - vezbe', 'racunari', 3], 
                                 ['Funkcionalno programiranje', None, 2], ['Funkcionalno programiranje - vezbe', 'racunari', 3],
                                 ['Programiranje za veb', None, 2], ['Programiranje za veb - vezbe', 'racunari', 3],
                                 ['Istrazivanje podataka 2', None, 2], ['Istrazivanje podataka 2 - vezbe', 'racunari', 3],
                                 ['Razvoj sofvtvera', None, 2], ['Razvoj softvera - vezbe', 'racunari', 3],
                                 ['Projektovanje baza podataka', None, 2], ['Projektovanje baza podataka - vezbe', 'racunari', 3],
                                ]), columns=['name', 'equipment', 'number_of_classes'])
classes

,name,equipment,number_of_classes
0,Racunarske mreze,None,2
1,Racunarske mreze - vezbe,racunari,3
2,Racunarska inteligencija,None,2
3,Racunarska inteligencija - vezbe,racunari,3
4,Funkcionalno programiranje,None,2
5,Funkcionalno programiranje - vezbe,racunari,3
6,Programiranje za veb,None,2
7,Programiranje za veb - vezbe,racunari,3
8,Istrazivanje podataka 2,None,2
9,Istrazivanje podataka 2 - vezbe,racunari,3


In [133]:
groups = pd.DataFrame(np.array([['4i', None], ['4i1a', '4i'], ['4i1b', '4i'], ['4i2a', '4i'], ['4i2b', '4i'],
                                ['4r', None], ['r4gr1', '4r'], ['r4gr2', '4r'], ['r4gr3', '4r']
                               ]), columns=['name', 'parent'])
groups

,name,parent
0,4i,None
1,4i1a,4i
2,4i1b,4i
3,4i2a,4i
4,4i2b,4i
5,4r,None
6,r4gr1,4r
7,r4gr2,4r
8,r4gr3,4r


In [17]:
locations = pd.DataFrame(np.array([['Studentski trg'], ['Sv. Nikole'], ['Jagic']]), columns=['name'])
locations

,name
0,Studentski trg
1,Sv. Nikole
2,Jagic


In [556]:
location_distances = pd.DataFrame(np.array([['0', '1', 1], ['0', '2', 1], ['1', '2', 1]]), columns=['first', 'second', 'hours'])
location_distances

,first,second,hours
0,0,1,1
1,0,2,1
2,1,2,1


In [477]:
rooms = pd.DataFrame(np.array([['704', '0', 'racunari'], ['718', '0', 'racunari'], ['706', '0', None], ['821', '0', None],
                               ['BIM', '0', 'racunari'], ['RLAB', '0', 'racunari'], ['830', '0', None], ['N201', '1', 'racunari'],
                               ['N202', '1', 'racunari'], ['N253', '1', None], ['Jag1', '2', 'racunari'],
                               ['Jag2', '2', 'racunari'], ['Jag3', '2', None], ['Jag4', '2', None]
                              ]), columns=['name', 'location', 'equipment'])
rooms

,name,location,equipment
0,704,0,racunari
1,718,0,racunari
2,706,0,None
3,821,0,None
4,BIM,0,racunari
5,RLAB,0,racunari
6,830,0,None
7,N201,1,racunari
8,N202,1,racunari
9,N253,1,None


In [97]:
teaching_curriculum = pd.DataFrame(np.array([['0', '0', {'0'}], ['0', '1', {'1', '2'}], ['0', '1', {'3', '4'}], ['3', '10', {'1', '2'}],
                                             ['3', '10', {'3', '4', '5'}], ['3', '11', {'1'}], ['3', '11', {'2'}], ['3', '11', {'3', '5'}],
                                             ['3', '11', {'4'}], ['2', '8', {'0'}], ['2', '9', {'1', '2'}], ['2', '9', {'3', '4'}]                                               
                                            ]), columns=['lecturer', 'class', 'groups'])
teaching_curriculum

,lecturer,class,groups
0,0,0,{0}
1,0,1,"{2, 1}"
2,0,1,"{4, 3}"
3,3,10,"{2, 1}"
4,3,10,"{4, 3, 5}"
5,3,11,{1}
6,3,11,{2}
7,3,11,"{5, 3}"
8,3,11,{4}
9,2,8,{0}


In [415]:
teaching_curriculum_copy = teaching_curriculum.copy()
teaching_curriculum_copy['lecturer'] = lecturers.iloc[teaching_curriculum_copy['lecturer'].tolist()].name.tolist()
teaching_curriculum_copy['class'] = classes.iloc[teaching_curriculum_copy['class'].tolist()].name.tolist()
teaching_curriculum_copy['groups'] = list(map(lambda gs: list(map(lambda g: groups.iloc[int(g)]['name'], gs)), teaching_curriculum_copy['groups'].tolist()))
teaching_curriculum_copy

,lecturer,class,groups
0,Kartelj,Racunarske mreze,[4i]
1,Kartelj,Racunarske mreze - vezbe,"[4i1b, 4i1a]"
2,Kartelj,Racunarske mreze - vezbe,"[4i2b, 4i2a]"
3,Malkov,Razvoj sofvtvera,"[4i1b, 4i1a]"
4,Malkov,Razvoj sofvtvera,"[4i2b, 4i2a, 4r]"
5,Malkov,Razvoj softvera - vezbe,[4i1a]
6,Malkov,Razvoj softvera - vezbe,[4i1b]
7,Malkov,Razvoj softvera - vezbe,"[4r, 4i2a]"
8,Malkov,Razvoj softvera - vezbe,[4i2b]
9,Mitic,Istrazivanje podataka 2,[4i]


In [158]:
time_slots = range(0, 60)

In [403]:
def init_timetable():
    init = []
    for i in range(teaching_curriculum.shape[0]):
        init.append([rooms.iloc[i % len(rooms)].name, time_slots[i // len(rooms)]])
    timetable = pd.DataFrame(np.array(init), columns = ['room', 'time_slot'])
    return timetable

In [1012]:
def are_classes_intersected(timetable,teaching_curriculum, i, j):
    first = timetable.iloc[i]['time_slot'] + classes.iloc[int(teaching_curriculum.iloc[i]['class'])]['number_of_classes'] <= timetable.iloc[j].time_slot
    second = timetable.iloc[j]['time_slot'] + classes.iloc[int(teaching_curriculum.iloc[j]['class'])]['number_of_classes'] <= timetable.iloc[i].time_slot
    return not (first or second)

def are_locations_intersected(timetable,teaching_curriculum, i, j):
    i_location = rooms.iloc[timetable.iloc[i]['room']]['location']
    j_location = rooms.iloc[timetable.iloc[j]['room']]['location']
    if (i_location == j_location) or ((timetable.iloc[i]['time_slot'] // 12) != (timetable.iloc[j]['time_slot'] // 12)):
        return False
    first_location = min(int(i_location), int(j_location))
    second_location = max(int(i_location), int(j_location))
    distance_in_hours = int(location_distances[(location_distances['first'] == str(first_location)) & (location_distances['second'] == str(second_location))].iloc[0]['hours'])
    first = abs(timetable.iloc[i]['time_slot'] + classes.iloc[int(teaching_curriculum.iloc[i]['class'])]['number_of_classes'] - timetable.iloc[j].time_slot) >= distance_in_hours 
    second = abs(timetable.iloc[j]['time_slot'] + classes.iloc[int(teaching_curriculum.iloc[j]['class'])]['number_of_classes'] - timetable.iloc[i].time_slot) >= distance_in_hours
    return not (first and second)

def are_groups_intersected(teaching_curriculum, i, j):
    set1 = teaching_curriculum.iloc[i]['groups']
    set2 = teaching_curriculum.iloc[j]['groups']
    for el1 in set1:
        for el2 in set2:
            if el1 == el2 or groups.iloc[int(el1)]['name'] == groups.iloc[int(el2)]['parent'] or groups.iloc[int(el2)]['name'] == groups.iloc[int(el1)]['parent']:
                return True
    return False

In [1011]:
def calculate_soft_constraints(timetable, teaching_curriculum, indexes):
    list_of_timeslots = [set() for x in range(60)]
    for i in indexes:
        timeslot = timetable.iloc[i]['time_slot']
        num_of_classes = classes.iloc[int(teaching_curriculum.iloc[i]['class'])]['number_of_classes']
        for j in range(timeslot, timeslot + num_of_classes):
            list_of_timeslots[j].add(rooms.iloc[timetable.iloc[i]['room']]['location'])
    count = 0
    last_set = set()
    last_index = 0
    for i in range(len(list_of_timeslots)):
        if i % 12 == 0:
            last_set = set()
        if last_set and last_index < i - 1 and list_of_timeslots[i]:
            count += 1
        if last_set and list_of_timeslots[i] and last_set.isdisjoint(list_of_timeslots[i]):
            count += 1
        if (list_of_timeslots[i]):
            last_set = list_of_timeslots[i]
            last_index = i
    return count

def calculate_lecturer_preferences(timetable, teaching_curriculum, indexes):
    list_of_timeslots = [False for x in range(60)]
    list_of_locations = set()
    lecturer = lecturers.iloc[int(teaching_curriculum.iloc[indexes[0]]['lecturer'])]
    for i in indexes:
        timeslot = timetable.iloc[i]['time_slot']
        num_of_classes = classes.iloc[int(teaching_curriculum.iloc[i]['class'])]['number_of_classes']
        list_of_locations.add(rooms.iloc[timetable.iloc[i]['room']]['location'])
        for j in range(timeslot, timeslot + num_of_classes):
            list_of_timeslots[j] = True
    count = 0
    if lecturer['preffered_locations'] and list_of_locations.difference(lecturer['preffered_locations']):
        count += 1
    for i in range(len(lecturer['not_preffered_timeslots'])):
        if list_of_timeslots[lecturer['not_preffered_timeslots'][i]]:
            count += 1
    if lecturer['preffered_timeslots']:
        for i in range(len(list_of_timeslots)):
            if list_of_timeslots[i] and i not in lecturer['preffered_timeslots']:
                count += 1
    return count

In [1004]:
def lecturer_constraint(timetable, teaching_curriculum):
    count = 0
    for i in range(teaching_curriculum.shape[0]):
        for j in range(i+1, teaching_curriculum.shape[0]):
            if teaching_curriculum.iloc[i]['lecturer'] == teaching_curriculum.iloc[j]['lecturer'] and (are_classes_intersected(timetable, teaching_curriculum, i, j) or are_locations_intersected(timetable, teaching_curriculum, i, j)):
                count += 1
    return count

def lecturer_constraint_delta(timetable, teaching_curriculum, i):
    lecturer = teaching_curriculum.iloc[i]['lecturer']
    teaching_curriculum_with_lecturer = teaching_curriculum[teaching_curriculum['lecturer'] == lecturer]
    count = 0
    for j in teaching_curriculum_with_lecturer.index:
        if i != j:
            if (are_classes_intersected(timetable, teaching_curriculum, i, j) or are_locations_intersected(timetable, teaching_curriculum, i, j)):
                count += 1
    return count

In [1005]:
def group_constraint(timetable, teaching_curriculum):
    count = 0
    for i in range(teaching_curriculum.shape[0]):
        for j in range(i+1, teaching_curriculum.shape[0]):
            if are_groups_intersected(teaching_curriculum, i, j) and (are_classes_intersected(timetable, teaching_curriculum, i, j) or are_locations_intersected(timetable, teaching_curriculum, i, j)):
                count += 1
    return count

def group_constraint_delta(timetable, teaching_curriculum, i):
    count = 0
    for j in range(teaching_curriculum.shape[0]):
        if i != j:
            if are_groups_intersected(teaching_curriculum, i, j) and (are_classes_intersected(timetable, teaching_curriculum, i, j) or are_locations_intersected(timetable, teaching_curriculum, i, j)):
                count += 1
    return count

In [1006]:
def room_constraint(timetable, teaching_curriculum):
    count = 0
    for i in range(teaching_curriculum.shape[0]):
        for j in range(i+1, teaching_curriculum.shape[0]):
            if timetable.iloc[i]['room'] == timetable.iloc[j]['room'] and are_classes_intersected(timetable, teaching_curriculum, i, j):
                count += 1
    return count

def room_constraint_delta(timetable, teaching_curriculum, i):
    room = timetable.iloc[i]['room']
    timetable_with_room = timetable[timetable['room'] == room]
    count = 0
    for j in timetable_with_room.index:
        if i != j:
            if are_classes_intersected(timetable, teaching_curriculum, i, j):
                count += 1
    return count

In [1007]:
def equipment_constraint(timetable, teaching_curriculum):
    count = 0
    for i in range(teaching_curriculum.shape[0]):
        if classes.iloc[int(teaching_curriculum.iloc[i]['class'])]['equipment'] is not None and classes.iloc[int(teaching_curriculum.iloc[i]['class'])]['equipment'] != rooms.iloc[int(timetable.iloc[i]['room'])]['equipment']:
            count += 1
    return count

def equipment_constraint_delta(timetable, teaching_curriculum, i):
    count = 0
    if classes.iloc[int(teaching_curriculum.iloc[i]['class'])]['equipment'] is not None and classes.iloc[int(teaching_curriculum.iloc[i]['class'])]['equipment'] != rooms.iloc[int(timetable.iloc[i]['room'])]['equipment']:
        count += 1
    return count

In [1013]:
def lecturer_soft_constraint(timetable, teaching_curriculum):
    count = 0
    for i in lecturers.index:
        indexes = teaching_curriculum[teaching_curriculum['lecturer'] == str(i)].index
        if indexes.empty:
            continue
        count += calculate_soft_constraints(timetable, teaching_curriculum, indexes)
        count += calculate_lecturer_preferences(timetable, teaching_curriculum, indexes)
    return count

def lecturer_soft_constraint_delta(timetable, teaching_curriculum, i):
    lecturer_index = teaching_curriculum.iloc[i]['lecturer']
    indexes = teaching_curriculum[teaching_curriculum['lecturer'] == lecturer_index].index
    return calculate_soft_constraints(timetable, teaching_curriculum, indexes) + calculate_lecturer_preferences(timetable, teaching_curriculum, indexes)

In [1014]:
def groups_soft_constraint(timetable, teaching_curriculum):
    count = 0
    for i in groups.index:
        if groups.iloc[i]['parent']:
            indexes = teaching_curriculum[(teaching_curriculum['groups'].astype(str).str.contains(str(i))) | (teaching_curriculum['groups'].astype(str).str.contains(str(groups[groups['name'] == groups.iloc[i]['parent']].index[0])))].index
            count += calculate_soft_constraints(timetable, teaching_curriculum, indexes)
    return count

def groups_soft_constraint_delta(timetable, teaching_curriculum, i):
    group_set = teaching_curriculum.iloc[i]['groups']
    count = 0
    for el in group_set:
        if groups.iloc[int(el)]['parent']:
            indexes = teaching_curriculum[(teaching_curriculum['groups'].astype(str).str.contains(el)) | (teaching_curriculum['groups'].astype(str).str.contains(str(groups[groups['name'] == groups.iloc[int(el)]['parent']].index[0])))].index
            count += calculate_soft_constraints(timetable, teaching_curriculum, indexes)
        else:
            children_groups = groups[groups['parent'] == groups.iloc[int(el)]['name']].index
            for j in children_groups:
                indexes = teaching_curriculum[(teaching_curriculum['groups'].astype(str).str.contains(str(j))) | (teaching_curriculum['groups'].astype(str).str.contains(str(groups[groups['name'] == groups.iloc[j]['parent']].index[0])))].index
                count += calculate_soft_constraints(timetable, teaching_curriculum, indexes)
    return count

In [1008]:
def calculate_fittnes(timetable, teaching_curriculum):
    first = lecturer_constraint(timetable, teaching_curriculum) + group_constraint(timetable, teaching_curriculum) + room_constraint(timetable, teaching_curriculum) + equipment_constraint(timetable, teaching_curriculum)
    second = lecturer_soft_constraint(timetable, teaching_curriculum)+ groups_soft_constraint(timetable, teaching_curriculum)
    return (first, second)

def calculate_fitness_delta(timetable, teaching_curriculum, i):
    first = lecturer_constraint_delta(timetable, teaching_curriculum, i) + group_constraint_delta(timetable, teaching_curriculum, i) + room_constraint_delta(timetable, teaching_curriculum, i) + equipment_constraint_delta(timetable, teaching_curriculum, i)
    second = lecturer_soft_constraint_delta(timetable, teaching_curriculum, i) + groups_soft_constraint_delta(timetable, teaching_curriculum, i)
    return (first, second)

In [411]:
def local_search(timetable, teaching_curriculum):
    best_timetable = timetable.copy()
    best_fitness = calculate_fittnes(timetable, teaching_curriculum)
    while(True):
        for i in range(timetable.shape[0]):
            timeslot_range = range(12 * (timetable.iloc[i]['time_slot'] // 12), 12 * (timetable.iloc[i]['time_slot'] // 12 + 1) - classes.iloc[int(teaching_curriculum.iloc[i]['class'])]['number_of_classes'] + 1)
            for timeslot in timeslot_range:
                current_timetable = timetable.copy()
                current_timetable.iloc[i]['time_slot'] = timeslot
                current_fittness = calculate_fittnes(current_timetable, teaching_curriculum)
                if current_fittness < best_fitness:
                    best_timetable = current_timetable.copy()
                    best_fitness = current_fittness
            for j in range(rooms.shape[0]):
                current_timetable = timetable.copy()
                current_timetable.iloc[i]['room'] = rooms.iloc[j].name
                current_fittness = calculate_fittnes(current_timetable, teaching_curriculum)
                if current_fittness < best_fitness:
                    best_timetable = current_timetable.copy()
                    best_fitness = current_fittness
        if best_fitness < calculate_fittnes(timetable, teaching_curriculum):
            print(best_fitness)
            timetable = best_timetable.copy()
        else:
            break
    return timetable

In [1009]:
def local_search_v2(timetable, teaching_curriculum):
    best_timetable = timetable.copy()
    best_fitness = calculate_fittnes(timetable, teaching_curriculum)
    fitness = best_fitness
    while(True):
        for i in range(timetable.shape[0]):
            timeslot_range = range(12 * (timetable.iloc[i]['time_slot'] // 12), 12 * (timetable.iloc[i]['time_slot'] // 12 + 1) - classes.iloc[int(teaching_curriculum.iloc[i]['class'])]['number_of_classes'] + 1)
            for timeslot in timeslot_range:
                current_timetable = timetable.copy()
                current_timetable.iloc[i]['time_slot'] = timeslot
                fitness_delta = calculate_fitness_delta(timetable, teaching_curriculum, i)
                current_fitness_delta = calculate_fitness_delta(current_timetable, teaching_curriculum, i)
                current_fittness = (fitness[0] - fitness_delta[0] + current_fitness_delta[0], fitness[1] - fitness_delta[1] + current_fitness_delta[1])
                if current_fittness[0] < best_fitness[0] or (current_fittness[0] == best_fitness[0] and current_fittness[1] < best_fitness[1]):
                    best_timetable = current_timetable.copy()
                    best_fitness = current_fittness
            for j in range(rooms.shape[0]):
                current_timetable = timetable.copy()
                current_timetable.iloc[i]['room'] = rooms.iloc[j].name
                fitness_delta = calculate_fitness_delta(timetable, teaching_curriculum, i)
                current_fitness_delta = calculate_fitness_delta(current_timetable, teaching_curriculum, i)
                current_fittness = (fitness[0] - fitness_delta[0] + current_fitness_delta[0], fitness[1] - fitness_delta[1] + current_fitness_delta[1])
                if current_fittness[0] < best_fitness[0] or (current_fittness[0] == best_fitness[0] and current_fittness[1] < best_fitness[1]):
                    best_timetable = current_timetable.copy()
                    best_fitness = current_fittness
        if best_fitness[0] < fitness[0] or (best_fitness[0] == fitness[0] and best_fitness[1] < fitness[1]):
            print(best_fitness)
            timetable = best_timetable.copy()
            fitness = best_fitness
        else:
            break
    return timetable

In [685]:
def shaking(timetable, teaching_curriculum):
    time_table_shaking = timetable.copy()
    for j in range(k):
        random_timetable_index = random.choice(range(timetable.shape[0]))
        random_room_index = random.choice(range(rooms.shape[0]))
        random_timeslot = random.choice(time_slots)
        while(random_timeslot % 12 + classes.iloc[int(teaching_curriculum.iloc[random_timetable_index]['class'])]['number_of_classes'] > 11):
            random_timeslot = random.choice(time_slots)
        time_table_shaking.iloc[random_timetable_index] = np.array([[random_room_index, random_timeslot]])
    return time_table_shaking

In [999]:
k = 40
result_timetable = init_timetable()
print('init', calculate_fittnes(result_timetable, teaching_curriculum))
for i in range(10):
    print('iteration', i)
    timetable = result_timetable.copy()
    timetable = shaking(timetable, teaching_curriculum)
    timetable = local_search_v2(timetable, teaching_curriculum)
    current_fitness = calculate_fittnes(timetable, teaching_curriculum)
    current_best_fitness = calculate_fittnes(result_timetable, teaching_curriculum)
    if current_fitness[0] < current_best_fitness[0] or (current_fitness[0] == current_best_fitness[0] and current_fitness[1] < current_best_fitness[1]):
        result_timetable = timetable

init (62, 2)
iteration 0
(8, 17)
(6, 19)
(5, 17)
(4, 17)
(3, 17)
(2, 19)
(2, 18)
(2, 17)
(2, 16)
(2, 15)
(2, 14)
(1, 17)
iteration 1
(6, 35)
(5, 34)
(4, 33)
(3, 33)
(2, 33)
(1, 34)
(0, 36)
(0, 33)
(0, 31)
(0, 30)
(0, 29)
(0, 28)
(0, 27)
(0, 26)
(0, 25)
(0, 24)
iteration 2
(11, 23)
(8, 33)
(7, 33)
(6, 33)
(5, 33)
(4, 33)
(3, 33)
(2, 35)
(1, 39)
(0, 43)
(0, 39)
(0, 36)
(0, 33)
(0, 31)
(0, 29)
(0, 27)
(0, 25)
(0, 24)
(0, 23)
(0, 22)
iteration 3
(5, 26)
(4, 26)
(3, 26)
(2, 26)
(1, 26)
(0, 26)
(0, 24)
(0, 22)
(0, 20)
(0, 18)
(0, 17)
iteration 4
(6, 33)
(4, 30)
(3, 30)
(2, 30)
(1, 30)
(0, 30)
(0, 27)
(0, 24)
(0, 22)
(0, 20)
(0, 18)
(0, 17)
iteration 5
(5, 42)
(4, 40)
(3, 39)
(2, 38)
(1, 37)
(0, 43)
(0, 40)
(0, 38)
(0, 36)
(0, 35)
(0, 34)
(0, 33)
(0, 32)
(0, 31)
iteration 6
(5, 36)
(4, 35)
(3, 35)
(2, 36)
(1, 43)
(0, 43)
(0, 39)
(0, 35)
(0, 32)
(0, 29)
(0, 27)
(0, 25)
(0, 24)
iteration 7
(6, 17)
(4, 20)
(3, 19)
(2, 19)
(1, 19)
(0, 23)
(0, 18)
(0, 14)
(0, 12)
(0, 11)
(0, 10)
(0, 9)
iteration 8

In [1000]:
result_timetable

,room,time_slot
0,0,3
1,8,13
2,5,5
3,2,26
4,6,53
5,7,16
6,0,5
7,0,50
8,1,8
9,2,45


In [646]:
teaching_curriculum_copy

,lecturer,class,groups
0,Kartelj,Racunarske mreze,[4i]
1,Kartelj,Racunarske mreze - vezbe,"[4i1b, 4i1a]"
2,Kartelj,Racunarske mreze - vezbe,"[4i2b, 4i2a]"
3,Malkov,Razvoj sofvtvera,"[4i1b, 4i1a]"
4,Malkov,Razvoj sofvtvera,"[4i2b, 4i2a, 4r]"
5,Malkov,Razvoj softvera - vezbe,[4i1a]
6,Malkov,Razvoj softvera - vezbe,[4i1b]
7,Malkov,Razvoj softvera - vezbe,"[4r, 4i2a]"
8,Malkov,Razvoj softvera - vezbe,[4i2b]
9,Mitic,Istrazivanje podataka 2,[4i]


In [975]:
lecturers

,name,preffered_timeslots,not_preffered_timeslots,preffered_locations
0,Kartelj,"[0, 1, 2, 3, 4, 5]","[6, 7, 8, 9, 10, 11]",{2}
1,Cukic,[],[],{}
2,Mitic,[],[],{}
3,Malkov,"[6, 7, 8, 9, 10, 11]","[12, 13, 14, 15, 16, 17]",{0}


In [1001]:
calculate_fittnes(result_timetable, teaching_curriculum)

(0, 9)